In [1]:
import copy

from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import get_kraken_orderbook, get_omnipool_data, get_orderbooks_from_file, get_binance_orderbook
from hydradx.model.processing import get_omnipool_data_from_file
from hydradx.model.amm.centralized_market import CentralizedMarket
import hydradx.model.amm.arbitrage_agent as arb

from pprint import pprint

import json


Set `get_live_markets = True` to get live markets from Kraken. Set to `False` to use order books from file.

In [2]:
get_live_markets = False

In [3]:
# 2: Acala DAI
# 3: Acala WBTC
# 4: Acala WETH
# 5: DOT
# 9: ASTR
# 10: native USDT
# 11: iBTC
# 12: ZTG
# 13: CFG
# 14: BNC
# 15: vDOT
# 16: GLMR
# 17: INTR
# 18: Wormhole DAI
# 19: Wormhole WBTC
# 20: Wormhole WETH
# 100: 4-Pool

In [4]:
RPC="wss://rpc.hydradx.cloud"
            
if get_live_markets:
    asset_list, asset_map, tokens, fees = get_omnipool_data(RPC, archive=True)
else:
    asset_list, asset_map, tokens, fees = get_omnipool_data_from_file("data/input/")
    
pprint(tokens)


{'4-Pool': {'LRNA': 46343.07135572106, 'liquidity': 1332687.383283113},
 'ASTR': {'LRNA': 10119.257861197138, 'liquidity': 4184895.5795255746},
 'BNC': {'LRNA': 6881.816147019611, 'liquidity': 700204.5495188688},
 'CFG': {'LRNA': 10185.106861296503, 'liquidity': 538980.8722571266},
 'DAI': {'LRNA': 12343.469085393072, 'liquidity': 353398.13980324025},
 'DAI001': {'LRNA': 1407.281258338137, 'liquidity': 37585.53464434876},
 'DOT': {'LRNA': 70446.59338482533, 'liquidity': 392562.1774897939},
 'GLMR': {'LRNA': 5856.589118748921, 'liquidity': 607889.0886780379},
 'HDX': {'LRNA': 22096.92769782686, 'liquidity': 103887831.85203914},
 'INTR': {'LRNA': 9071.548819526934, 'liquidity': 12632175.69695167},
 'USDT': {'LRNA': 81323.50064917025, 'liquidity': 2333794.900523},
 'WBTC': {'LRNA': 41081.475714824424, 'liquidity': 31.63776176},
 'WBTC001': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WETH': {'LRNA': 22022.6498604935, 'liquidity': 300.89764978733405},
 'WETH001': {'LRNA': 73063.05

In [5]:
# arb_file = "arbconfig.txt"
arb_file = "arbconfig2.txt"
with open('config/' + arb_file, 'r') as json_file:
    cfg = json.load(json_file)
    
for d in cfg:
    d['tkns'] = tuple(d['tkns'])
    d['tkn_ids'] = tuple(d['tkn_ids'])
    d['order_book'] = tuple(d['order_book'])
    


ob_objs = {}
order_book_assets = {}
buffer = {}

if get_live_markets:
    for arb_cfg in cfg:
        tkn_pair = arb_cfg['order_book']
        exchange = arb_cfg['exchange']
        if exchange not in ob_objs:
            ob_objs[exchange] = {}
        if tkn_pair not in ob_objs[exchange]:
            if exchange == 'kraken':
                ob_objs[exchange][tkn_pair] = get_kraken_orderbook(tkn_pair, archive=True)
            elif exchange == 'binance':
                ob_objs[exchange][tkn_pair] = get_binance_orderbook(tkn_pair, archive=True)
            else:
                raise ValueError("Exchange not supported")
else:
    ob_objs = get_orderbooks_from_file("data/input/")

for arb_cfg in cfg:
    tkn_pair = arb_cfg['order_book']
    exchange = arb_cfg['exchange']
    for tkn in tkn_pair:
        if exchange not in order_book_assets:
            order_book_assets[exchange] = []
        if tkn not in order_book_assets[exchange]:
            order_book_assets[exchange].append(tkn) 
            
    arb_cfg['tkn_pair'] = (asset_map[arb_cfg['tkn_ids'][0]], asset_map[arb_cfg['tkn_ids'][1]])
            

                


In [6]:

cex_fees = {
    'kraken': 0.0016,
    'binance': 0.0010
}

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0
# asset_fee = 0.0

cex_dict = {}
for exchange in ob_objs:
    cex_dict[exchange] = CentralizedMarket(
        order_book=ob_objs[exchange],
        asset_list=order_book_assets[exchange],
        trade_fee=cex_fees[exchange]
    )

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)
init_state = op_state.copy()


In [7]:
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state1 = op_state.copy()
all_swaps1 = arb.get_arb_swaps_simple(op_state1, cex_dict1, cfg)
# print("strategy 1")
# pprint(all_swaps1)
# pprint(len(all_swaps1))

cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
op_state2 = op_state.copy()
all_swaps2 = arb.get_arb_swaps(op_state2, cex_dict2, cfg)
print("strategy 2")
pprint(all_swaps2)
pprint(len(all_swaps2))

asset_limit = 500

cex_dict3 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
liq_file = "liqconfig.txt"
with open('config/' + liq_file, 'r') as json_file:
    max_liquidity = json.load(json_file)
        
op_state3 = op_state.copy()
all_swaps3 = arb.get_arb_swaps(op_state3, cex_dict3, cfg, max_liquidity=max_liquidity)
print("strategy 3")
pprint(all_swaps3)
pprint(len(all_swaps3))

cex_dict4 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
max_liquidity4 = copy.deepcopy(max_liquidity)
op_state4 = op_state.copy()
# asset_file = "assetconfig.txt"
asset_file = "assetconfigzeroes.txt"
with open('config/' + asset_file, 'r') as json_file:
    asset_cfg = json.load(json_file)
op_agent = Agent(holdings={asset: 1000000000 for asset in op_state4.asset_list}, unique_id='omnipool')
kraken_agent = Agent(holdings={asset: 1000000000 for asset in cex_dict4['kraken'].asset_list}, unique_id='kraken')
binance_agent = Agent(holdings={asset: 1000000000 for asset in cex_dict4['binance'].asset_list}, unique_id='binance')
agent_dict = {'omnipool': op_agent, 'kraken': kraken_agent, 'binance': binance_agent}
all_swaps4 = arb.get_arb_swaps_inventory(op_state4, cex_dict4, agent_dict, cfg, asset_cfg, max_liquidity=max_liquidity4)
print("strategy 4")
pprint(all_swaps4)
pprint(len(all_swaps4))

strategy 2
[{'cex': {'amount': 0.00803,
          'buy_asset': 'DAI',
          'price': 37126.24759,
          'sell_asset': 'BTC',
          'trade': 'sell'},
  'dex': {'amount': 0.00803,
          'buy_asset': 'WBTC',
          'max_sell': 281.62287445521355,
          'price': 37144.82,
          'sell_asset': 'DAI001',
          'trade': 'buy'},
  'exchange': 'binance'},
 {'cex': {'amount': 0.024698872846603395,
          'buy_asset': 'DAI',
          'price': 37126.237595,
          'sell_asset': 'BTC',
          'trade': 'sell'},
  'dex': {'amount': 0.024698872846603395,
          'buy_asset': 'WBTC',
          'max_sell': 894.2355517555475,
          'price': 37144.81,
          'sell_asset': 'DAI001',
          'trade': 'buy'},
  'exchange': 'binance'},
 {'cex': {'amount': 0.651,
          'buy_asset': 'USD',
          'price': 0.28585699999999997,
          'sell_asset': 'BNC',
          'trade': 'sell'},
  'dex': {'amount': 0.651,
          'buy_asset': 'BNC',
          'max

In [8]:
state1 = init_state.copy()
state2 = init_state.copy()
state3 = init_state.copy()
state4 = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list}
for exchange in cex_dict:
    for asset in cex_dict[exchange].asset_list:
        if asset not in holdings:
            holdings[asset] = 1000000000
init_agent = Agent(holdings=holdings, unique_id='bot')
agent1 = init_agent.copy()
agent2 = init_agent.copy()
agent3 = init_agent.copy()
agent4 = init_agent.copy()
cex_dict1 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict2 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict3 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}
cex_dict4 = {exchange: cex_dict[exchange].copy() for exchange in cex_dict}

arb.execute_arb(state1, cex_dict1, agent1, all_swaps1)
arb.execute_arb(state2, cex_dict2, agent2, all_swaps2)
arb.execute_arb(state3, cex_dict3, agent3, all_swaps3)
arb.execute_arb(state4, cex_dict4, agent4, all_swaps4)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'iBTC': 'BTC',
             'XBT': 'BTC',}

profit1 = arb.calculate_profit(init_agent, agent1, asset_map)
profit2 = arb.calculate_profit(init_agent, agent2, asset_map)
profit3 = arb.calculate_profit(init_agent, agent3, asset_map)
profit4 = arb.calculate_profit(init_agent, agent4, asset_map)

asset_map_omnipool = {
    'ETH': 'WETH',
    'XETH': 'WETH',
    'XXBT': 'WBTC',
    'BTC': 'WBTC',
    'ZUSD': 'USDT',
    'USD': 'USDT',
    'USDC': 'USDT',
    'DAI': 'USDT',
    'USDT001': 'USDT',
    'DAI001': 'USDT',
    'WETH001': 'WETH',
    'WBTC001': 'WBTC',
    'iBTC': 'WBTC',
    'XBT': 'WBTC',
}


profit1_omnipool = arb.calculate_profit(init_agent, agent1, asset_map_omnipool)
profit2_omnipool = arb.calculate_profit(init_agent, agent2, asset_map_omnipool)
profit3_omnipool = arb.calculate_profit(init_agent, agent3, asset_map_omnipool)
profit4_omnipool = arb.calculate_profit(init_agent, agent4, asset_map_omnipool)

print("strategy 1 profit")
print(profit1)
print("strategy 1 profit at Omnipool spot")
print(sum([profit1_omnipool[asset] * state1.price(state1, asset, 'USDT') for asset in profit1_omnipool]))
print("strategy 2 profit")
print(profit2)
print("strategy 2 profit at Omnipool spot")
print(sum([profit2_omnipool[asset] * state2.price(state2, asset, 'USDT') for asset in profit2_omnipool]))
print("strategy 3 profit")
print(profit3)
print("strategy 3 profit at Omnipool spot")
print(sum([profit3_omnipool[asset] * state3.price(state3, asset, 'USDT') for asset in profit3_omnipool]))
print("strategy 4 profit")
print(profit4)
print("strategy 4 profit at Omnipool spot")
print(sum([profit4_omnipool[asset] * state4.price(state4, asset, 'USDT') for asset in profit4_omnipool]))


strategy 1 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 42.970832109451294, 'ETH': 0.0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 1 profit at Omnipool spot
42.97083210945129
strategy 2 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 47.406232714653015, 'ETH': 0.0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 2 profit at Omnipool spot
47.406232714653015
strategy 3 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 45.08865261077881, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0.0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 3 profit at Omnipool spot
45.0886526107788
strategy 4 profit
{'4-Pool': 0, 'HDX': 0, 'USD': 39.2230486869812, 'ETH': 0, 'GLMR': 0, 'BTC': 0.0, 'BNC': 0.0, 'vDOT': 0, 'CFG': 0, 'DOT': 0, 'ZTG': 0, 'INTR': 0, 'ASTR': 0}
strategy 4 profit at Omnipool spot
39.2230486869812


In [9]:
for swap in all_swaps4:
    if 'max_sell' in swap['dex']:
        profit = swap['cex']['price']*swap['cex']['amount'] - swap['dex']['max_sell']
        pprint(swap['dex']['buy_asset'] + ' in for ' + swap['dex']['sell_asset'] + ' out')
        num_asset = swap['dex']['sell_asset']
        pprint(str(swap['cex']['price']*swap['cex']['amount']) + ' ' + num_asset)
        pprint('profit: ' + str(profit) + ' ' + num_asset)
        print()
    elif 'min_buy' in swap['dex']:
        profit = swap['dex']['min_buy'] - swap['cex']['price']*swap['cex']['amount']
        pprint(swap['dex']['buy_asset'] + ' in for ' + swap['dex']['sell_asset'] + ' out')
        num_asset = swap['dex']['buy_asset']
        pprint(str(swap['cex']['price']*swap['cex']['amount']) + ' ' + num_asset)
        pprint('profit: ' + str(profit) + ' ' + num_asset)
        print()

'WBTC in for DAI001 out'
'298.1237681477 DAI001'
'profit: 16.36015262579224 DAI001'

'WBTC in for DAI001 out'
'916.9656000839773 DAI001'
'profit: 22.29377029827276 DAI001'

'BNC in for USDT out'
'0.186092907 USDT'
'profit: 0.001734910774642967 USDT'
